In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


In [ ]:
# Load the dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
data = pd.read_csv(url, usecols=[1])
data = data.values.astype('float32')

# Visualize the data
plt.figure(figsize=(10, 6))
plt.plot(data, label='Air Passengers')
plt.title('Monthly Total of International Airline Passengers')
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.legend()
plt.show()

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 12
X, y = create_sequences(data_normalized, seq_length)

# Convert to TensorFlow tensors
X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [ ]:
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(seq_length, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
epochs = 100
history = model.fit(X_train, y_train, epochs=epochs, batch_size=1, validation_data=(X_test, y_test))


In [ ]:
# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform the predictions and actual values
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train_inv, train_predict))
test_rmse = np.sqrt(mean_squared_error(y_test_inv, test_predict))
print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')


In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(y_test_inv, label='Actual Data')
plt.plot(test_predict, label='Predicted Data')
plt.title('Air Passengers Prediction')
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.legend()
plt.show()


In [ ]:
def predict_future(model, data, future_periods, seq_length):
    predictions = data[-seq_length:].tolist()
    for _ in range(future_periods):
        seq = np.array(predictions[-seq_length:]).reshape((1, seq_length, 1))
        prediction = model.predict(seq)
        predictions.append([prediction[0, 0]])
    return predictions[seq_length:]


# Predict future values
future_periods = 12
predictions = predict_future(model, data_normalized, future_periods, seq_length)
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(scaler.inverse_transform(data_normalized), label='Historical Data')
plt.plot(np.arange(len(data_normalized), len(data_normalized) + future_periods), predictions, label='Future Predictions')
plt.title('Air Passengers Future Prediction')
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.legend()
plt.show()
